In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms

from PIL import Image

device = "cpu"
print(device)

cpu


In [39]:
# preprocessing

# loading image
image = Image.open('1.jpg')
image = image.resize((120,120))

#image.show()

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((120,120)),
    transforms.Normalize(0.5,0.5,0.5)
])

image = trans(image)
print(image.shape)
image = image.view(1,3,120,120)

maxp = torch.nn.MaxPool2d(2,2)
convex = torch.nn.Conv2d(3, 8, kernel_size=3)
convex1 = torch.nn.Conv2d(8, 16, kernel_size=3)

batch1 = torch.nn.BatchNorm2d(8)
batch2 = torch.nn.BatchNorm2d(16)

a = batch2(convex1(maxp(batch1(convex(image)))))
print(a.shape)

convtrans = torch.nn.ConvTranspose2d(16,8,kernel_size=2, stride=1)
convtrans1 = torch.nn.ConvTranspose2d(8,3,kernel_size=1, stride=1)

b = convtrans1(convtrans(a))
print(b.shape)


torch.Size([3, 120, 120])
torch.Size([1, 16, 57, 57])
torch.Size([1, 3, 58, 58])


In [ ]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

# SKIP Connection
#print(image.shape)
#image = image.view(32, 3, 120, 120)
#print(image.shape)

# UnetAE class definition

class unetAE(nn.Module):
  def __init__(self):
    super(unetAE, self).__init__()

    self.linearf = nn.Sequential(
      nn.Linear(36864, 7600),
      nn.Linear(7600, 1600),
      nn.Linear(1600, 600),
      nn.Linear(600, 256),
      nn.Linear(256, 128)
      )

    self.linearb = nn.Sequential(
      nn.Linear(128,256),
      nn.Linear(256,600),
      nn.Linear(600,1600),
      nn.Linear(1600, 7600),
      nn.Linear(7600, 36864),
      )

    self.down_conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
    self.down_conv2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
    self.down_conv3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
    self.down_conv4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
    self.up_conv1 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=1),
            nn.ReLU(inplace=True),
        )
    self.up_conv2 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=1),
            nn.ReLU(inplace=True),
        )
    self.up_conv3 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=1),
            nn.ReLU(inplace=True),
        )
    self.up_conv4 = nn.Sequential(
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=1),
            nn.ReLU(inplace=True),
        )
  


  def forward(self, x):
    
    print(x.shape, "fuck")
    x = self.down_conv1(x)
    x = self.down_conv2(x)
    x = self.down_conv3(x)
    x = self.down_conv4(x)
    print(x.shape, 'shape')
    x = x.view(x.size(0), -1)

    print(x.shape, "yes")
    out = self.linearf(x)

    out = self.linearb(out)

    print(out.shape, "rebuild linear shape")

    out = out.view(32,256,12,12)

    print(out.shape, "rebuild linear shape 1")
    out = self.up_conv1(out)
    print(out.shape, "rebuild linear shape 1.5")
    out = self.up_conv2(out)
    print(out.shape, "rebuild linear shape 2")
    out = self.up_conv3(out)
    print(out.shape, "rebuild linear shape 3")
    out = self.up_conv4(out)
    print(out.shape, "rebuild linear shape 2")

    

    return out

model2 = unetAE()
model2.to(device)

image = image.to(device)
print(type(image))

#x = model2(image)

In [36]:
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import os

class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.transform = trans
        self.image_filenames = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        image_path = os.path.join(self.root_dir, self.image_filenames[index])
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

dataset = CustomDataset('D:/tempmassstorage/train2014/train2014')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

a = iter(dataloader)
b = next(a)
print(b)

tensor([[[[-5.1990e-01, -4.9520e-01, -4.7797e-01,  ..., -4.1141e-01,
           -4.1569e-01, -4.1781e-01],
          [-4.9703e-01, -4.9020e-01, -4.7984e-01,  ..., -4.0931e-01,
           -4.0392e-01, -4.0931e-01],
          [-4.9150e-01, -4.8480e-01, -4.7533e-01,  ..., -3.9984e-01,
           -4.0000e-01, -4.0686e-01],
          ...,
          [ 2.8121e-01,  3.5588e-01,  3.3922e-01,  ...,  6.0997e-01,
            7.2353e-01,  7.7925e-01],
          [ 5.7170e-01,  5.8049e-01,  7.0467e-01,  ...,  7.0739e-01,
            6.6039e-01,  6.8853e-01],
          [ 4.3536e-01,  6.0627e-01,  5.8908e-01,  ...,  6.0608e-01,
            5.0088e-01,  5.5843e-01]],

         [[-1.2775e-01, -1.0696e-01, -9.3660e-02,  ..., -1.1405e-02,
           -1.5686e-02, -1.7810e-02],
          [-1.0595e-01, -1.0196e-01, -9.5523e-02,  ..., -9.3136e-03,
           -3.9216e-03, -9.3141e-03],
          [-1.0588e-01, -1.0049e-01, -8.3170e-02,  ...,  1.6332e-04,
            0.0000e+00, -6.8628e-03],
          ...,
     

In [41]:
# hyper parameters for training
model =unetAE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
lossfn = torch.nn.MSELoss()
epoch = 100

test = torch.nn.KLDivLoss()

for i in range(1):
    for image in dataloader:
        generated = model(image)

        print(generated.shape, " generated")
        print(image.shape, " image")

        print(generated.type, " generated")
        print(image.type, " image")

        loss = lossfn(generated, image)

        loss.backward()
        print(f'loss: {loss.item()}, epoch: {i}')

        optimizer.step()

        optimizer.zero_grad()

        break


torch.Size([32, 3, 120, 120]) fuck
torch.Size([32, 256, 12, 12]) shape
torch.Size([32, 36864]) yes
torch.Size([32, 36864]) rebuild linear shape
torch.Size([32, 256, 12, 12]) rebuild linear shape 1
torch.Size([32, 128, 24, 24]) rebuild linear shape 1.5
torch.Size([32, 64, 48, 48]) rebuild linear shape 2
torch.Size([32, 32, 96, 96]) rebuild linear shape 3
torch.Size([32, 3, 192, 192]) rebuild linear shape 2
torch.Size([32, 3, 192, 192])  generated
torch.Size([32, 3, 120, 120])  image
<built-in method type of Tensor object at 0x00000222F231D300>  generated
<built-in method type of Tensor object at 0x000002219CD407C0>  image


c:\Users\deshp\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32, 3, 120, 120])) that is different to the input size (torch.Size([32, 3, 192, 192])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (192) must match the size of tensor b (120) at non-singleton dimension 3

In [ ]:
# scheduler

import torch.nn.functional as F

def linear_beta_scheduler(timestamps=None, start=0.0001, end=0.02):
    return torch.linspace(start, end, timestamps)

#print(torch.linspace(0.00001, 0.02, 100))
a = torch.tensor([0.01])
print(torch.rand_like(a))

alpha = 1 - (linear_beta_scheduler(Time=10))

alpha_cum_prod = torch.prod(alpha)

tensor([0.4673])


In [40]:
print(x.shape)

x = x.view(3,240,240)

image = transforms.ToPILImage()(x)
image.show()


NameError: name 'x' is not defined